In [35]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import pandas_bokeh
from qgrid import show_grid
# import finplot as fplt

import polygon_rest_api as papi
import polygon_backfill as pb
import polygon_s3 as ps3
import bar_samples as bs
import bar_labels as bl

import filters as ft
# import market_cluster as mc

import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20, 7]
from matplotlib import style
style.use('fivethirtyeight')

# from tqdm import tqdm
# import ipdb
# ipdb.set_trace(context=10)

# df.style.background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)  # 'Blues', 'coolwarm', 'RdBu_r' & 'BrBG' are other good diverging colormaps
# pd.set_option('precision', 2)

# get local market daily data
# mdf = pd.read_parquet('data/mdf.parquet')

# .plot_bokeh(sizing_mode="scale_height")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
symbol='GLD'
# symbol = 'market'
# date = '2020-01-02'
start_date = '2020-01-01'
# start_date = '2020-10-01'
# end_date = '2020-01-10'
# symbols=['SPY', 'QQQ']
tick_type='trades'

result_path = '/Users/bobcolner/QuantClarity/pandas-polygon/data'

thresh = {
    # GT thresholds
    'renko_reveral_multiple': 2,
    'duration_sec': 60*45,
    # LT thresholds 
    'min_duration_sec': 10,
    'min_tick_count': 30,
}

In [3]:
import ray

@ray.remote
def build_bars_ray(result_path, symbol, date, thresh) -> dict:
    # get ticks for current date
    ticks_df = ps3.load_ticks(result_path, symbol, date, 'trades')
    # sample bars
    bars, state = bs.build_bars(ticks_df, thresh)
    return {'date': date, 'bars': bars}


In [ ]:
ray.init(ignore_reinit_error=True)

daily_vol_df = ps3.get_symbol_vol_filter(result_path, symbol, start_date)
daily_vol_df.loc[:, 'range_jma_lag'] = daily_vol_df.range_jma.shift(1)
daily_vol_df = daily_vol_df.dropna()

futures = []

for row in daily_vol_df.itertuples():
    
    thresh.update({'renko_size': row.range_jma_lag / 15})

    bars = build_bars_ray.remote(
        result_path=result_path,
        symbol=symbol, 
        date=row.date,
        thresh=thresh
    )
    futures.append(bars)
    
bar_dates = ray.get(futures)

ray.shutdown()

In [ ]:
results = []
for date_d in bar_dates:
    
    imbalances = []    
    for bar in date_d['bars']:
        imbalances.append(bar['tick_imbalance'])
    
    imbal_thresh = pd.Series(imbalances).quantile(q=.95)
    results.append({'date': date_d['date'], 'bar_count': len(date_d['bars']), 'tick_imbalance_thresh': imbal_thresh})

daily_bar_stats_df = ft.jma_filter_df(pd.DataFrame(results), 'tick_imbalance_thresh', length=5, power=1)
daily_bar_stats_df.loc[:,'tick_imbalance_thresh_jma_lag'] = daily_bar_stats_df['tick_imbalance_thresh_jma'].shift(1)
daily_bar_stats_df = daily_bar_stats_df.dropna()
daily_join_df = pd.merge(left=daily_bar_stats_df, right=daily_vol_df, left_on='date', right_on='date')

# daily_join_df[['tick_imbalance_thresh','tick_imbalance_thresh_jma']].plot()

In [ ]:
ray.init(ignore_reinit_error=True)

futures = []

for row in daily_join_df.itertuples():
    
    thresh.update({
        'renko_size': row.range_jma_lag / 15,
        'tick_imbalance': row.tick_imbalance_thresh_jma_lag
        })

    bars = build_bars_ray.remote(
        result_path=result_path,
        symbol=symbol, 
        date=row.date,
        thresh=thresh
    )
    futures.append(bars)
    
bar_dates_3 = ray.get(futures)

ray.shutdown()

In [49]:
bars_df = pd.DataFrame(bar_dates_2[0]['bars'])
# bars_df = pd.DataFrame(bar_dates[0]['bars'])

In [50]:
abs(bars_df.tick_imbalance).describe()

count    210.000000
mean      40.428571
std       32.799334
min        0.000000
25%        7.000000
50%       40.500000
75%       74.000000
max      170.000000
Name: tick_imbalance, dtype: float64

In [67]:
pd.DataFrame(bar_dates_3[130]['bars']).bar_trigger.value_counts()

tick_imbalance    37
renko_down        30
renko_up          26
duration           6
Name: bar_trigger, dtype: int64

In [55]:
pd.Series([len(date['bars']) for date in bar_dates_3]).describe()

count    204.000000
mean     113.049020
std       58.059317
min       37.000000
25%       72.750000
50%       96.000000
75%      134.000000
max      426.000000
dtype: float64

In [52]:
pd.Series([len(date['bars']) for date in bar_dates_2]).describe()

count    204.000000
mean     156.357843
std       79.291036
min       40.000000
25%      100.000000
50%      132.500000
75%      190.250000
max      477.000000
dtype: float64

In [51]:
pd.Series([len(date['bars']) for date in bar_dates]).describe()

count    205.000000
mean      88.897561
std       52.049413
min       33.000000
25%       56.000000
50%       75.000000
75%      106.000000
max      383.000000
dtype: float64

In [ ]:
results = []
for date_d in bar_dates:
    
    imbalances = []    
    for bar in date_d['bars']:
        imbalances.append(bar['tick_imbalance'])
    
    imbal_q80 = pd.Series(imbalances).quantile(q=.8)
    results.append({'date': date_d['date'], 'bar_count': len(date_d['bars']), 'tick_imbalance_p80': imbal_q80})


In [ ]:
show_grid(pd.DataFrame(bar_dates[open_dates[2]])[['bar_trigger','open_at','close_at','duration_min','ticks','price_wmean']])

In [ ]:
# range / price = HFT profit potential
(df.price.max() - df.price.min()) / df.price.mean()

In [ ]:
# label bars
label_horizon_mins = 30
risk_level = 0.2
# reward_base = 1
reward_ratios = np.arange(3, 20, .5)

lbars = bl.label_bars(bars, ticks_df, risk_level, label_horizon_mins, reward_ratios)

lbars_df = pd.DataFrame(lbars)

lbars_unq = bl.get_concurrent_stats(lbars_df)
print(lbars_unq['grand_avg_unq'])

In [ ]:
lbars_df[['price_wmean', 'label_rrr']].plot_bokeh(sizing_mode="scale_height")
# 17, 31

In [ ]:
lbars_df[['price_wmean', 'label_rrr']].plot(subplots=True, figsize=[17,12])

In [ ]:
df[['range', 'range_jma']].plot()
# df2[['range', 'range_jma']].plot_bokeh(sizing_mode="scale_height")